In [2]:
import numpy as np
import pandas as pd
from scipy.stats import entropy
import os
import itertools as it



PATTERN_MATRIX_FILE = os.path.join('./' ,"pattern_matrix.npy")

PATTERN_GRID_DATA = dict()


In [3]:
def get_word_list(short=False):
    result = list()
    if short:
        file = 'possible_words.txt'
    else:
        file = 'allowed_words.txt'

    with open(file) as fp:
        result.extend([word.strip() for word in fp.readlines()])
    return result

def get_word_freq():
    freq_dict = dict()
    file = 'short_freqs.txt'

    with open(file) as fp:
        for line in fp.readlines():
            pieces = line.split(' ')
            word = pieces[0]
            freq = pieces[1].strip()
            freq_dict.update({word : freq})
    file = 'long_freqs.txt'
    with open(file) as fp:
        for line in fp.readlines():
                pieces = line.split(' ')
                word = pieces[0]
                freq = pieces[1].strip()
                freq_dict.update({word : freq})

    return freq_dict

In [4]:
words = get_word_list()
words[:10]

['aahed',
 'aalii',
 'aargh',
 'aarti',
 'abaca',
 'abaci',
 'aback',
 'abacs',
 'abaft',
 'abaka']

In [5]:
word_freqs = get_word_freq()
word_freqs

{'aback': '0.0000009348',
 'abase': '0.0000000921725',
 'abate': '0.00000153374',
 'abbey': '0.00000202411',
 'abbot': '0.00000214039',
 'abhor': '0.000000567927',
 'abide': '0.00000436668',
 'abled': '0.0000000846074',
 'abode': '0.00000386656',
 'abort': '0.000000594533',
 'about': '0.000988835',
 'above': '0.000280052',
 'abuse': '0.0000257216',
 'abyss': '0.00000176845',
 'acorn': '0.000000488927',
 'acrid': '0.00000070628',
 'actor': '0.0000098911',
 'acute': '0.000026595',
 'adage': '0.000000758717',
 'adapt': '0.00000673097',
 'adept': '0.00000140932',
 'admin': '0.000000833169',
 'admit': '0.0000256365',
 'adobe': '0.000000826063',
 'adopt': '0.0000224539',
 'adore': '0.0000013532',
 'adorn': '0.00000141946',
 'adult': '0.0000352473',
 'affix': '0.0000009154',
 'afire': '0.000000311371',
 'afoot': '0.000000698031',
 'afoul': '0.000000286726',
 'after': '0.000641221',
 'again': '0.000256395',
 'agape': '0.000000325656',
 'agate': '0.000000600712',
 'agent': '0.000060683',
 'agil

In [6]:
import enum
import collections

class Tip(enum.Enum):
    ABSENT = 0
    PRESENT = 1
    CORRECT = 2

def score(secret, guess):
    # All characters that are not correct go into the usable pool.
    pool = collections.Counter(s for s, g in zip(secret, guess) if s != g)
    # Create a first tentative score by comparing char by char.
    score = []
    for secret_char, guess_char in zip(secret, guess):
        if secret_char == guess_char:
            score.append(Tip.CORRECT)
        elif guess_char in secret and pool[guess_char] > 0:
            score.append(Tip.PRESENT)
            pool[guess_char] -= 1
        else:
            score.append(Tip.ABSENT)

    return score

def filter_words(words, guess, score):
    """Filter words to only keep those that respect the score for the given guess."""

    new_words = []
    for word in words:
        # The pool of characters that account for the PRESENT ones is all the characters
        # that do not correspond to CORRECT positions.
        pool = collections.Counter(c for c, sc in zip(word, score) if sc != Tip.CORRECT)
        for char_w, char_g, sc in zip(word, guess, score):
            if sc == Tip.CORRECT and char_w != char_g:
                break  # Word doesn't have the CORRECT character.
            elif char_w == char_g and sc != Tip.CORRECT:
                break  # If the guess isn't CORRECT, no point in having equal chars.
            elif sc == Tip.PRESENT:
                if not pool[char_g]:
                    break  # Word doesn't have this PRESENT character.
                pool[char_g] -= 1
            elif sc == Tip.ABSENT and pool[char_g]:
                break  # ABSENT character shouldn't be here.
        else:
            new_words.append(word)  # No `break` was hit, so store the word.

    return new_words

def guess_vis(score):
    """ Prints guess with respect to answer like Wordle """
    vis_str = str()
    for i in score:
        if i == Tip.ABSENT:
            vis_str += "\U00002B1B" # Black Square Unicode
        elif i == Tip.PRESENT:
            vis_str += "\U0001F7E8" # Yellow Sqaure Unicode
        else:
            vis_str += "\U0001F7E9" # Green Square Unicode

    print(vis_str)

def sigmoid(x):
    import math
    sig = 1 / (1 + math.exp(-x))
    return sig

def get_frequency_based_priors(n_common=3000, width_under_sigmoid=10):
    """
    We know that that list of wordle answers was curated by some human
    based on whether they're sufficiently common. This function aims
    to associate each word with the likelihood that it would actually
    be selected for the final answer.
    Sort the words by frequency, then apply a sigmoid along it.
    """
    freq_map = get_word_freq()
    words = np.array(list(freq_map.keys()))
    freqs = np.array([freq_map[w] for w in words])
    arg_sort = freqs.argsort()
    sorted_words = words[arg_sort]

    # We want to imagine taking this sorted list, and putting it on a number
    # line so that it's length is 10, situating it so that the n_common most common
    # words are positive, then applying a sigmoid
    x_width = width_under_sigmoid
    c = x_width * (-0.5 + n_common / len(words))
    xs = np.linspace(c - x_width / 2, c + x_width / 2, len(words))
    priors = dict()
    for word, x in zip(sorted_words, xs):
        priors[word] = sigmoid(x)
    return priors

def get_weights(words, priors):
    frequencies = np.array([priors[word] for word in words])
    total = frequencies.sum()
    if total == 0:
        return np.zeros(frequencies.shape)
    return frequencies / total

def generate_pattern_grid(words1, words2):
    """
    A pattern for two words represents the worle-similarity
    pattern (grey -> 0, yellow -> 1, green -> 2) but as an integer
    between 0 and 3^5. Reading this integer in ternary gives the
    associated pattern.
    This function computes the pairwise patterns between two lists
    of words, returning the result as a grid of hash values. Since
    this is the most time consuming part of many computations, all
    operations that can be are vectorized, perhaps at the expense
    of easier readibility.
    """
    # Convert word lists to integer arrays
    w1, w2 = (
        np.array([[ord(c) for c in w] for w in words], dtype=np.uint8)
        for words in (words1, words2)
    )

    if len(w1) == 0 or len(w2) == 0:
        return np.zeros((len(w1), len(w2)), dtype=np.uint8)

    # equality_grid[a, b, i, j] represents whether the ith letter
    # of words1[a] equals the jth letter of words2[b]
    equality_grid = np.zeros((len(w1), len(w2), 5, 5), dtype=bool)
    for i, j in it.product(range(5), range(5)):
        equality_grid[:, :, i, j] = np.equal.outer(w1[:, i], w2[:, j])

    patterns = np.zeros((len(w1), len(w2)), dtype=np.uint8)
    three_pows = (3**np.arange(5)).astype(np.uint8)
    for i, tp in enumerate(three_pows):
        # This accounts for yellow squares
        patterns[:, :] += tp * equality_grid[:, :, i, :].any(2)
        # This accounts for green squares
        patterns[:, :] += tp * equality_grid[:, :, i, i]

    return patterns


def generate_full_pattern_grid():
    words = get_word_list()
    grid = generate_pattern_grid(words, words)
    np.save(PATTERN_MATRIX_FILE, grid)


def get_pattern_grid(words1, words2):
    if not PATTERN_GRID_DATA:
        if not os.path.exists(PATTERN_MATRIX_FILE):
            generate_full_pattern_grid()
        PATTERN_GRID_DATA['grid'] = np.load(PATTERN_MATRIX_FILE)
        PATTERN_GRID_DATA['words_to_index'] = dict(zip(
            get_word_list(), it.count()
        ))

    full_grid = PATTERN_GRID_DATA['grid']
    words_to_index = PATTERN_GRID_DATA['words_to_index']

    indices1 = [words_to_index[w] for w in words1]
    indices2 = [words_to_index[w] for w in words2]
    return full_grid[np.ix_(indices1, indices2)]


def get_possible_words(guess, pattern, word_list):
    all_hashes = get_pattern_grid([guess], word_list).flatten()
    return list(np.array(word_list)[all_hashes == pattern])


def get_word_buckets(guess, possible_words):
    buckets = [[] for x in range(3**5)]
    hashes = get_pattern_grid([guess], possible_words).flatten()
    for index, word in zip(hashes, possible_words):
        buckets[index].append(word)
    return buckets


def get_pattern_distributions(allowed_words, possible_words, weights):
    """
    For each possible guess in allowed_words, this finds the probability
    distribution across all of the 3^5 wordle patterns you could see, assuming
    the possible answers are in possible_words with associated probabilities
    in weights.
    It considers the pattern hash grid between the two lists of words, and uses
    that to bucket together words from possible_words which would produce
    the same pattern, adding together their corresponding probabilities.
    """
    pattern_matrix = get_pattern_grid(allowed_words, possible_words)

    n = len(allowed_words)
    distributions = np.zeros((n, 3**5))
    n_range = np.arange(n)
    for j, prob in enumerate(weights):
        distributions[n_range, pattern_matrix[:, j]] += prob
    return distributions


def entropy_of_distributions(distributions, atol=1e-12):
    axis = len(distributions.shape) - 1
    return entropy(distributions, base=2, axis=axis)

def get_entropies(allowed_words, possible_words, weights):
    if weights.sum() == 0:
        return np.zeros(len(allowed_words))
    distributions = get_pattern_distributions(allowed_words, possible_words, weights)
    return entropy_of_distributions(distributions)

def sort_entropies(allowed_words, possible_words, weights):
    sorted_words = dict()

    entropies = get_entropies(allowed_words, possible_words, weights)
    arg_sort = (-entropies).argsort()
    sorted_entropies = np.array(entropies)[arg_sort]
    temp_words = np.array(allowed_words)[arg_sort]

    for x in range(len(temp_words)):
        sorted_words.update({temp_words[x] : sorted_entropies[x]})

    return sorted_words

# Solvers

def basic_solver(guess=None):
    from numpy import random
    from time import sleep

    all_words = get_word_list()
    allowed_words = all_words
    possible_words = all_words
    ans_list = get_word_list(short=True)
    answer = ans_list[random.randint(len(ans_list))]
    weights = get_weights(all_words, get_frequency_based_priors())

    if guess == None:
        guess = 'crane'
    else:
        guess = guess

    guesses = [guess]
    guess_entropy = sort_entropies(allowed_words, possible_words, weights)[guess]
    print(f'The correct wordle is: {answer}')

    while guess != answer:
        print(f"Current Guess: {guess} with {guess_entropy:.2f} bits of information")
        print(guess_vis(score(answer, guess)))
        filter = filter_words(allowed_words, guess, score(answer, guess))
        sorted_words = sort_entropies(filter, possible_words, weights)


        topthree = list(sorted_words.keys())[:2]
        guess = topthree[0]
        guess_entropy = sorted_words[guess]
        guesses.append(guess)
        allowed_words = filter
        print(f'Top guesses now are : {topthree}.')

        sleep(1)
    
    print(f"Current Guess: {guess}")
    print(guess_vis(score(answer, guess)))

def play_wordle():
    from numpy import random

    allowed_words = get_word_list()
    ans_list = get_word_list(short=True)
    answer = ans_list[random.randint(len(ans_list))]
    guess = ''

    while True:
        guess = input("What is your guess? \n")
        if guess not in allowed_words:
            print('Word not in list')
            
        elif guess == answer:
            print(guess_vis(score(answer, guess)))
            print("You got it!")
            return
        else:
            print(guess)
            print(guess_vis(score(answer, guess)))

def num_to_Tip(guess_score):
    ans = list()

    for x in guess_score:
        if x == 0:
            ans.append(Tip.ABSENT)
        elif x == 1:
            ans.append(Tip.PRESENT)
        elif x == 2:
            ans.append(Tip.CORRECT)
        else:
            print("Unknown Score input, try again.")
            break
    
    return ans
            
def wordle_helper():
    from time import sleep
    all_words = get_word_list()
    allowed_words = all_words
    possible_words = all_words
    weights = get_weights(all_words, get_frequency_based_priors())

    while True:
        guess = input("What was your guess?")
        print("What was the score?\n")
        first = int(input("First Letter Score"))
        second = int(input("Second Letter Score"))
        third = int(input("Third Letter Score"))
        fourth = int(input("Fourth Letter Score"))
        fifth = int(input("Fifth Letter Score"))
        guess_score = [first, second, third, fourth, fifth]
        guess_score = num_to_Tip(guess_score)

        filter = filter_words(allowed_words, guess, guess_score)
        sorted_words = sort_entropies(filter, possible_words, weights)
        topthree = list(sorted_words.keys())[:10]
        allowed_words = filter
        print(f"The top guesses are: {topthree}")
        sleep(1)


In [10]:
basic_solver()

The correct wordle is: brain
Current Guess: crane with 5.58 bits of information
⬛🟩🟩🟨⬛
None
Top guesses now are : ['train', 'drain'].
Current Guess: train with 5.42 bits of information
⬛🟩🟩🟩🟩
None
Top guesses now are : ['drain', 'grain'].
Current Guess: drain with 5.22 bits of information
⬛🟩🟩🟩🟩
None
Top guesses now are : ['grain', 'brain'].
Current Guess: grain with 4.95 bits of information
⬛🟩🟩🟩🟩
None
Top guesses now are : ['brain'].
Current Guess: brain
🟩🟩🟩🟩🟩
None


In [5]:
wordle_helper()

What was the score?

The top guesses are: ['riots', 'tiros', 'tirls', 'roils', 'loirs', 'loris', 'rotis', 'rotls', 'dorts', 'dirts']
What was the score?

The top guesses are: ['lordy', 'world', 'loury', 'gourd', 'bourd', 'pudor', 'duroy', 'furol', 'dormy', 'fluor']
What was the score?

The top guesses are: ['humor']
What was the score?

The top guesses are: ['humor']
What was the score?

